# Activity 2 - SELECT JOIN

In [1]:
import sqlite3
db = sqlite3.connect('eu_football_database.sqlite')

### Basic Joins

In [3]:
cursor = db.execute("""
    SELECT * FROM League INNER JOIN Country
    ON League.country_id = Country.id
""")

In [4]:
cursor.fetchall()

[(1, 1, 'Belgium Jupiler League', 1, 'Belgium'),
 (1729, 1729, 'England Premier League', 1729, 'England'),
 (4769, 4769, 'France Ligue 1', 4769, 'France'),
 (7809, 7809, 'Germany 1. Bundesliga', 7809, 'Germany'),
 (10257, 10257, 'Italy Serie A', 10257, 'Italy'),
 (13274, 13274, 'Netherlands Eredivisie', 13274, 'Netherlands'),
 (15722, 15722, 'Poland Ekstraklasa', 15722, 'Poland'),
 (17642, 17642, 'Portugal Liga ZON Sagres', 17642, 'Portugal'),
 (19694, 19694, 'Scotland Premier League', 19694, 'Scotland'),
 (21518, 21518, 'Spain LIGA BBVA', 21518, 'Spain'),
 (24558, 24558, 'Switzerland Super League', 24558, 'Switzerland')]

In [5]:
cursor = db.execute("""
    SELECT League.name, Country.name FROM League INNER JOIN Country
    ON League.country_id = Country.id
""")

In [6]:
cursor.fetchall()

[('Belgium Jupiler League', 'Belgium'),
 ('England Premier League', 'England'),
 ('France Ligue 1', 'France'),
 ('Germany 1. Bundesliga', 'Germany'),
 ('Italy Serie A', 'Italy'),
 ('Netherlands Eredivisie', 'Netherlands'),
 ('Poland Ekstraklasa', 'Poland'),
 ('Portugal Liga ZON Sagres', 'Portugal'),
 ('Scotland Premier League', 'Scotland'),
 ('Spain LIGA BBVA', 'Spain'),
 ('Switzerland Super League', 'Switzerland')]

**Aliases**

In [7]:
cursor = db.execute("""
    SELECT L.name, C.name FROM League L INNER JOIN Country C
    ON L.country_id = C.id
""")

In [8]:
cursor.fetchall()

[('Belgium Jupiler League', 'Belgium'),
 ('England Premier League', 'England'),
 ('France Ligue 1', 'France'),
 ('Germany 1. Bundesliga', 'Germany'),
 ('Italy Serie A', 'Italy'),
 ('Netherlands Eredivisie', 'Netherlands'),
 ('Poland Ekstraklasa', 'Poland'),
 ('Portugal Liga ZON Sagres', 'Portugal'),
 ('Scotland Premier League', 'Scotland'),
 ('Spain LIGA BBVA', 'Spain'),
 ('Switzerland Super League', 'Switzerland')]

### Prettier Output using Pandas

In [9]:
import pandas as pd

In [10]:
pd.read_sql("""
    SELECT L.name, C.name FROM League L INNER JOIN Country C
    ON L.country_id = C.id
""", db)

,name,name
0,Belgium Jupiler League,Belgium
1,England Premier League,England
2,France Ligue 1,France
3,Germany 1. Bundesliga,Germany
4,Italy Serie A,Italy
5,Netherlands Eredivisie,Netherlands
6,Poland Ekstraklasa,Poland
7,Portugal Liga ZON Sagres,Portugal
8,Scotland Premier League,Scotland
9,Spain LIGA BBVA,Spain


We'll use "Real Madrid" in the following queries. Just in case, this is its basic info:

In [13]:
pd.read_sql('SELECT * FROM Team WHERE team_long_name = "Real Madrid CF"', db)

,id,team_api_id,team_fifa_api_id,team_long_name,team_short_name
0,43040,8633,243,Real Madrid CF,REA


### Advanced Join
_(3 or more tables)_

**The first match played by Real Madrid at home**

In [21]:
pd.read_sql("""
    SELECT
        C.name, L.name, M.id, M.date, T.team_long_name, M.away_team_api_id,
        M.home_team_goal, M.away_team_goal
    FROM Match M
    INNER JOIN Country C ON (M.country_id = C.id)
    INNER JOIN League L ON (M.league_id = L.id)
    INNER JOIN Team T ON (M.home_team_api_id = T.team_api_id)
    WHERE T.team_long_name = "Real Madrid CF"
    LIMIT 1""", db)

,name,name,id,date,team_long_name,away_team_api_id,home_team_goal,away_team_goal
0,Spain,Spain LIGA BBVA,21534,2008-11-08 00:00:00,Real Madrid CF,9864,4,3


Seems like they won 4-3. Who's the away team?

In [17]:
pd.read_sql("""
    SELECT *
    FROM Team T
    WHERE T.team_api_id = 9864
    LIMIT 1""", db)

,id,team_api_id,team_fifa_api_id,team_long_name,team_short_name
0,43054,9864,573,Málaga CF,MAL


This 2008 page seems to confirm it: https://resultados.as.com/resultados/futbol/primera/2008_2009/directo/regular_a_10_13538

**Could we have displayed both team names with only one query?**

Yes, you can join many times the same table:

In [20]:
pd.read_sql("""
    SELECT
        C.name, L.name, M.id, M.date, T_Home.team_long_name, T_Away.team_long_name,
        M.home_team_goal, M.away_team_goal
    FROM Match M
    INNER JOIN Country C ON (M.country_id = C.id)
    INNER JOIN League L ON (M.league_id = L.id)
    INNER JOIN Team T_Home ON (M.home_team_api_id = T_Home.team_api_id)
    INNER JOIN Team T_Away ON (M.away_team_api_id = T_Away.team_api_id)
    WHERE T_Home.id = 43040
    LIMIT 1""", db)

,name,name,id,date,team_long_name,team_long_name,home_team_goal,away_team_goal
0,Spain,Spain LIGA BBVA,21534,2008-11-08 00:00:00,Real Madrid CF,Málaga CF,4,3


So we could start also getting the initial team lineup:

In [28]:
pd.read_sql("""
    SELECT
        M.id, M.date, T_Home.team_long_name, T_Away.team_long_name,
        M.home_team_goal, M.away_team_goal,
        P_Home_1.player_name 'H1'
    FROM Match M
    INNER JOIN Country C ON (M.country_id = C.id)
    INNER JOIN League L ON (M.league_id = L.id)
    INNER JOIN Team T_Home ON (M.home_team_api_id = T_Home.team_api_id)
    INNER JOIN Team T_Away ON (M.away_team_api_id = T_Away.team_api_id)
    
    INNER JOIN Player P_Home_1 ON (M.home_player_1 = P_Home_1.player_api_id)
    
    WHERE T_Home.id = 43040
    LIMIT 1""", db)

,id,date,team_long_name,team_long_name,home_team_goal,away_team_goal,H1
0,21534,2008-11-08 00:00:00,Real Madrid CF,Málaga CF,4,3,Iker Casillas


According to [the report from 2008](https://resultados.as.com/resultados/futbol/primera/2008_2009/directo/regular_a_10_13538), seems like Iker was Real Madrid's Goalkeeper.

We could keep extending it:

In [30]:
pd.read_sql("""
    SELECT
        M.id, M.date, T_Home.team_long_name, T_Away.team_long_name,
        M.home_team_goal, M.away_team_goal,
        P_Home_1.player_name 'H1',
        P_Home_2.player_name 'H2'
    FROM Match M
    INNER JOIN Country C ON (M.country_id = C.id)
    INNER JOIN League L ON (M.league_id = L.id)
    INNER JOIN Team T_Home ON (M.home_team_api_id = T_Home.team_api_id)
    INNER JOIN Team T_Away ON (M.away_team_api_id = T_Away.team_api_id)
    
    INNER JOIN Player P_Home_1 ON (M.home_player_1 = P_Home_1.player_api_id)
    INNER JOIN Player P_Home_2 ON (M.home_player_2 = P_Home_2.player_api_id)
    
    WHERE T_Home.id = 43040
    LIMIT 1""", db)

,id,date,team_long_name,team_long_name,home_team_goal,away_team_goal,H1,H2
0,21534,2008-11-08 00:00:00,Real Madrid CF,Málaga CF,4,3,Iker Casillas,Sergio Ramos


**But we hate repetition, so we can make Python do the hard work:**

In [45]:
base_query = """
    SELECT
        M.id, M.date, T_Home.team_long_name, T_Away.team_long_name,
        M.home_team_goal, M.away_team_goal,
        
        {player_columns}
        
    FROM Match M
    INNER JOIN Country C ON (M.country_id = C.id)
    INNER JOIN League L ON (M.league_id = L.id)
    INNER JOIN Team T_Home ON (M.home_team_api_id = T_Home.team_api_id)
    INNER JOIN Team T_Away ON (M.away_team_api_id = T_Away.team_api_id)
    
    {player_joins}
    
    WHERE T_Home.id = 43040
    LIMIT 1"""

In [46]:
columns = ""
joins = ""
for i in range(1, 12):
    joins += ('INNER JOIN Player P_Home_{pos} ON '
              '(M.home_player_{pos} = P_Home_{pos}.player_api_id)\n').format(pos=i)
    columns += "P_Home_{pos}.player_name 'H{pos}',\n".format(pos=i)
columns = columns.rstrip(',\n')

In [47]:
print(joins)

INNER JOIN Player P_Home_1 ON (M.home_player_1 = P_Home_1.player_api_id)
INNER JOIN Player P_Home_2 ON (M.home_player_2 = P_Home_2.player_api_id)
INNER JOIN Player P_Home_3 ON (M.home_player_3 = P_Home_3.player_api_id)
INNER JOIN Player P_Home_4 ON (M.home_player_4 = P_Home_4.player_api_id)
INNER JOIN Player P_Home_5 ON (M.home_player_5 = P_Home_5.player_api_id)
INNER JOIN Player P_Home_6 ON (M.home_player_6 = P_Home_6.player_api_id)
INNER JOIN Player P_Home_7 ON (M.home_player_7 = P_Home_7.player_api_id)
INNER JOIN Player P_Home_8 ON (M.home_player_8 = P_Home_8.player_api_id)
INNER JOIN Player P_Home_9 ON (M.home_player_9 = P_Home_9.player_api_id)
INNER JOIN Player P_Home_10 ON (M.home_player_10 = P_Home_10.player_api_id)
INNER JOIN Player P_Home_11 ON (M.home_player_11 = P_Home_11.player_api_id)



In [48]:
print(columns)

P_Home_1.player_name 'H1',
P_Home_2.player_name 'H2',
P_Home_3.player_name 'H3',
P_Home_4.player_name 'H4',
P_Home_5.player_name 'H5',
P_Home_6.player_name 'H6',
P_Home_7.player_name 'H7',
P_Home_8.player_name 'H8',
P_Home_9.player_name 'H9',
P_Home_10.player_name 'H10',
P_Home_11.player_name 'H11'


In [49]:
query = base_query.format(player_joins=joins, player_columns=columns)

In [50]:
print(query)


    SELECT
        M.id, M.date, T_Home.team_long_name, T_Away.team_long_name,
        M.home_team_goal, M.away_team_goal,
        
        P_Home_1.player_name 'H1',
P_Home_2.player_name 'H2',
P_Home_3.player_name 'H3',
P_Home_4.player_name 'H4',
P_Home_5.player_name 'H5',
P_Home_6.player_name 'H6',
P_Home_7.player_name 'H7',
P_Home_8.player_name 'H8',
P_Home_9.player_name 'H9',
P_Home_10.player_name 'H10',
P_Home_11.player_name 'H11'
        
    FROM Match M
    INNER JOIN Country C ON (M.country_id = C.id)
    INNER JOIN League L ON (M.league_id = L.id)
    INNER JOIN Team T_Home ON (M.home_team_api_id = T_Home.team_api_id)
    INNER JOIN Team T_Away ON (M.away_team_api_id = T_Away.team_api_id)
    
    INNER JOIN Player P_Home_1 ON (M.home_player_1 = P_Home_1.player_api_id)
INNER JOIN Player P_Home_2 ON (M.home_player_2 = P_Home_2.player_api_id)
INNER JOIN Player P_Home_3 ON (M.home_player_3 = P_Home_3.player_api_id)
INNER JOIN Player P_Home_4 ON (M.home_player_4 = P_Home_4.playe

In [51]:
pd.read_sql(query, db)

,id,date,team_long_name,team_long_name,home_team_goal,away_team_goal,H1,H2,H3,H4,H5,H6,H7,H8,H9,H10,H11
0,21534,2008-11-08 00:00:00,Real Madrid CF,Málaga CF,4,3,Iker Casillas,Sergio Ramos,Fabio Cannavaro,Gabriel Heinze,Marcelo,Fernando Gago,Jose Maria Guti,Rafael van der Vaart,Wesley Sneijder,Royston Drenthe,Gonzalo Higuain
